# Star Resonance Tool - Colab Launcher

このノートブックは、Google Colab上で「Star Resonance Tool」を実行するためのものです。
以下のセルを順番に実行してください。

### 手順
1. 再生ボタン(▶)を押してセルを実行します。
2. 処理ログが表示されます。
3. 数秒〜数十秒待つと、ログの中に **`https://(ランダムな文字列).trycloudflare.com`** という形式のURLが表示されます。
4. そのリンクをクリックしてアプリを開いてください。

In [ ]:
# リポジトリのクローン
!git clone https://github.com/takeshich/star-resonance-tool.git
%cd star-resonance-tool

# 依存関係のインストール
!pip install -r requirements.txt

# cloudflared（トンネルツール）のダウンロード
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [ ]:
import subprocess
import time
import sys

# Streamlitをバックグラウンドで実行（CORS/XSRF保護を無効化）
print("Starting Streamlit...")
subprocess.Popen(['streamlit', 'run', 'app.py', '--server.enableCORS=false', '--server.enableXsrfProtection=false', '--server.port=8501'])
time.sleep(3)

# Cloudflare Tunnelを開始（ログを直接表示）
print("Starting Cloudflare Tunnel...")
print("Please wait for the URL (https://....trycloudflare.com) to appear below:")
print("="*50)

process = subprocess.Popen(['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8501'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# ログをリアルタイムで出力
try:
    for line in iter(process.stdout.readline, b''):
        line_str = line.decode('utf-8', errors='ignore')
        # URLが含まれる行を目立たせる（オプション）
        if 'trycloudflare.com' in line_str:
            print(f"\n >>> FOUND URL: {line_str.strip()} <<< \n")
        else:
            sys.stdout.write(line_str)
except KeyboardInterrupt:
    print("Stopped")
    process.terminate()